## Initialization

In [1]:
import os
import sys
from datetime import datetime

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split

import matplotlib.pyplot as plt
from tqdm import tqdm

root_dir = os.getcwd().split("AdversarialNIDS")[0] + "AdversarialNIDS"
sys.path.append(root_dir)

from scripts.logger import LoggerManager
from scripts.analysis.model_analysis import perform_model_analysis

from CICIDS2017.dataset import CICIDS2017
from UNSWNB15.dataset import UNSWNB15

from scripts.models.pytorch.MLP import NetworkIntrusionMLP
from scripts.models.pytorch.CNN import NetworkIntrustionCNN
from scripts.models.pytorch.LSTM import NetworkIntrusionLSTM

from scripts.models.pytorch.train import train
from scripts.models.pytorch.visualization import display_loss

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

lm = LoggerManager(
    log_dir=f"{root_dir}/results/logs",
    log_name="TDM",
)
logger = lm.get_logger()
title = lm.get_title()
logger.info(f"Logger initialized for '{title}'")
logger.info(f"Using device: {device}")

2025-11-25 11:33:32,647 - INFO - Logger initialized for 'TDM_20251125_113332'
2025-11-25 11:33:32,647 - INFO - Using device: cpu


## Initialization of the Dataset

In [4]:
full_dataset = UNSWNB15( # [UNSWNB15() or CICIDS2017()]
    dataset_size="small",
    logger=logger
).optimize_memory().encode(attack_encoder="label").scale(scaler="minmax")

2025-11-25 11:57:45,169 - INFO - Downloading dataset: mrwellsdavid/unsw-nb15
2025-11-25 11:57:49,480 - INFO - Loaded UNSW-NB15_1.csv with shape: (700000, 46)
2025-11-25 11:57:49,481 - INFO - DataFrame shape: (700000, 46)
2025-11-25 11:57:49,481 - INFO - Initial dimensions: 700,000 rows x 46 columns = 32,200,000 cells
2025-11-25 11:57:52,453 - INFO - ============================================================
2025-11-25 11:57:52,454 - INFO - Preprocessing completed successfully
2025-11-25 11:57:52,455 - INFO - Final dimensions: 640,658 rows x 46 columns
2025-11-25 11:57:52,455 - INFO - Total rows removed: 59,342 (8.48%)
2025-11-25 11:57:52,456 - INFO - data retention rate: 91.52%
2025-11-25 11:57:52,457 - INFO - ============================================================
2025-11-25 11:57:52,535 - INFO - Optimizing memory usage of the dataset...
2025-11-25 11:57:52,538 - INFO - Initial memory usage: 229.73 MB
2025-11-25 11:57:52,664 - INFO - Optimized memory usage: 136.86 MB
2025-11-25

In [4]:
dataset, multi_class = full_dataset.subset(size=50000, multi_class=False)

2025-11-22 16:24:01,934 - INFO - Subsetting dataset to size: 50000...
2025-11-22 16:24:01,944 - INFO - Class distribution before subsetting:
2025-11-22 16:24:01,945 - INFO -   Class 0: 1528113 samples
2025-11-22 16:24:01,945 - INFO -   Class 1: 414580 samples
2025-11-22 16:24:02,014 - INFO - Subsetted dataset to size: 50000


In [5]:
dataset.distribution()

2025-11-22 16:24:02,091 - INFO - Calculating data distribution...
2025-11-22 16:24:02,093 - INFO - Data Distribution by Attack Type:
2025-11-22 16:24:02,093 - INFO -   0: 25,000 instances
2025-11-22 16:24:02,093 - INFO -   1: 25,000 instances


0    25000
1    25000
Name: count, dtype: int64

In [ ]:
X_train, X_val, y_train, y_val = dataset.split(
    one_hot=True,
    apply_smote=True,
    to_tensor=True
)

In [ ]:
# Create DataLoaders
train_dataset = TensorDataset(X_train.to(device), y_train.to(device))
val_dataset = TensorDataset(X_val.to(device), y_val.to(device))

batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
input_size = train_loader.dataset.tensors[0].shape[1]
num_classes = train_loader.dataset.tensors[1].shape[1]
print(f"Input size: {input_size}, Num classes: {num_classes}")

model_type = f"{input_size}x{num_classes}"

criterion = nn.CrossEntropyLoss() if multi_class else nn.BCELoss()

## Multi Layers Perceptron (MLP)

In [ ]:
model_mlp = NetworkIntrusionMLP(input_size=input_size, num_classes=num_classes).to(device)
logger.info(f"MLP Model initialized with {model_mlp.num_params()} parameters")

learning_rate_mlp = 1e-2
num_epochs_mlp = 100

mlp_title = f"MLP_{model_type}_{num_epochs_mlp}"

optimizer_mlp = optim.AdamW(model_mlp.parameters(), lr=learning_rate_mlp)
scheduler_mlp = optim.lr_scheduler.ReduceLROnPlateau(optimizer_mlp, mode='min', factor=0.8, patience=10, min_lr=1e-6)

In [ ]:
model_mlp, train_losses_mlp, val_losses_mlp = train(
    model=model_mlp,
    optimizer=optimizer_mlp,
    scheduler=scheduler_mlp,
    criterion=criterion,
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=num_epochs_mlp,
    title=mlp_title,
    dir=f"{root_dir}/results/weights",
    logger=logger,
    device=device,
)

In [ ]:
display_loss(
    list_epoch_loss=train_losses_mlp,
    list_val_loss=val_losses_mlp,
    title=f"{title}_{mlp_title}",
    dir=f"{root_dir}/results/plots",
    plot=True,
    logger=logger,
    epoch_min=2
)

In [ ]:
cm, cr = perform_model_analysis(
    model=model_mlp,
    X_test=X_val,
    y_test=y_val,
    logger=logger,
    model_name=f"{title}_{mlp_title}",
    dir=f"{root_dir}/results/analysis",
    plot=True,
    device=device
)

model_mlp = model_mlp.cpu()

## Convolutional Neural Network (CNN)

In [ ]:
model_cnn = NetworkIntrustionCNN(input_channels=1, input_size= input_size, num_classes=num_classes).to(device)
logger.info(f"CNN Model initialized with {model_cnn.num_params()} parameters")

learning_rate_cnn = 1e-2
num_epochs_cnn = 100

cnn_title = f"CNN_{model_type}_{num_epochs_cnn}"

optimizer_cnn = optim.AdamW(model_cnn.parameters(), lr=learning_rate_cnn)
scheduler_cnn = optim.lr_scheduler.ReduceLROnPlateau(optimizer_cnn, mode='min', factor=0.8, patience=10, min_lr=1e-6)

In [ ]:
model_cnn, train_loss_cnn, val_loss_cnn = train(
    model=model_cnn,
    optimizer=optimizer_cnn,
    scheduler=scheduler_cnn,
    criterion=criterion,
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=num_epochs_cnn,
    title=cnn_title,
    dir=f"{root_dir}/results/weights",
    logger=logger,
    device=device,
)

In [ ]:
display_loss(
    train_loss_cnn,
    val_loss_cnn,
    title=f"{title}_{cnn_title}",
    dir=f"{root_dir}/results/plots",
    plot=True,
    logger=logger,
    epoch_min=2
)

In [ ]:
cm, cr = perform_model_analysis(
    model=model_cnn,
    X_test=X_val,
    y_test=y_val,
    model_name=f"{title}_{cnn_title}",
    dir=f"{root_dir}/results/analysis",
    plot=True,
    logger=logger,
    device=device
)

model_cnn = model_cnn.cpu()

## Long Short-Term Memory (LSTM)

In [ ]:
model_lstm = NetworkIntrusionLSTM(input_size=input_size, hidden_size=32, num_layers=2, num_classes=num_classes).to(device)
logger.info(f"LSTM Model initialized with {model_lstm.num_params()} parameters")

learning_rate_lstm = 1e-2
num_epochs_lstm = 100

lstm_title = f"LSTM_{model_type}_{num_epochs_lstm}"

optimizer_lstm = optim.AdamW(model_lstm.parameters(), lr=learning_rate_lstm)
scheduler_lstm = optim.lr_scheduler.ReduceLROnPlateau(optimizer_lstm, mode='min', factor=0.8, patience=10, min_lr=1e-6)

In [ ]:
model_lstm, train_loss_lstm, val_loss_lstm = train(
    model=model_lstm,
    optimizer=optimizer_lstm,
    scheduler=scheduler_lstm,
    criterion=criterion,
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=num_epochs_lstm,
    title=lstm_title,
    dir=f"{root_dir}/results/weights",
    logger=logger,
    device=device,
)

In [ ]:
display_loss(
    train_loss_lstm, 
    val_loss_lstm, 
    title=f"{title}_{lstm_title}",
    dir=f"{root_dir}/results/plots",
    plot=True,
    logger=logger,
    epoch_min=2
)

In [ ]:
cm, cr = perform_model_analysis(
    model=model_lstm,
    X_test=X_val,
    y_test=y_val,
    model_name=f"{title}_{lstm_title}",
    dir=f"{root_dir}/results/analysis",
    plot=True,
    logger=logger,
    device=device
)

# Move model to CPU to free up GPU memory
model_lstm = model_lstm.cpu()